In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"##last

# Data Aggregation and group operations

### Data Aggregation and group operations

- Split a pandas object into pieces using one or more keys (in the form of functions,
arrays, or DataFrame column names)
- Calculate group summary statistics, like count, mean, or standard deviation, or a
user-defined function
- Apply within-group transformations
- Compute pivot tables and cross-tabulations

###  General split-apply-combine

![](http://hadley.nz/hadley-wickham.jpg)

### split-apply-combine

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/81f9f.png)

- **Splitting** the data into groups based on some criteria
- **Applying** a function to each group independently: Aggregation,Transformation,Filtration
- **Combining** the results into a data structure

### Groupby Mechanics

- A list or array of values that is the same length as the axis being grouped
- A value indicating a column name in a DataFrame
- A dict or Series giving a correspondence between the values on the axis being
grouped and the group names
- A function to be invoked on the axis index or the individual labels in the index

### Data

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'], 
                   'data1' : np.random.randn(5), 
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.013507,-0.304265,a,one
1,-0.404252,-0.351571,a,two
2,0.312038,0.178852,b,one
3,-0.930780,-0.465142,b,two
4,-0.994283,-1.118791,a,one


### groupyby (keys)

In [86]:
grouped = df['data1'].groupby(df['key1'])
grouped
grouped.mean()

key1
a   -0.804014
b   -0.309371
Name: data1, dtype: float64

```
df['data1'].groupby(df['key1']) 
df[['data2']].groupby(df['key1'])
```

- Syntactic sugar


```
df.groupby('key1')['data1'] 
df.groupby('key1')[['data2']]
```


> 语法糖(Syntactic sugar),是由Peter J. Landin(和图灵一样的天才人物，是他最先发现了Lambda演算，由此而创立了函数式编程)创造的一个词语，它意指那些没有给计算机语言添加新功能，而只是对人类来说更“甜蜜”的语法。语法糖往往给程序员提供了更实用的编码方式，有益于更好的编码风格，更易读。不过其并没有给语言添加什么新东西。

In [84]:
grouped = df.groupby(df['key1'])['data1']##syntactic
grouped
grouped.mean()

key1
a   -0.804014
b   -0.309371
Name: data1, dtype: float64

In [85]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means
means.unstack()

key1  key2
a     one    -1.003895
      two    -0.404252
b     one     0.312038
      two    -0.930780
Name: data1, dtype: float64

key2,one,two
key1,,
a,-1.003895,-0.404252
b,0.312038,-0.930780


### groupyby (keys)

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/81f9f.png)

### Group by Series

In [41]:
df
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

,data1,data2,key1,key2
0,-1.013507,-0.304265,a,one
1,-0.404252,-0.351571,a,two
2,0.312038,0.178852,b,one
3,-0.930780,-0.465142,b,two
4,-0.994283,-1.118791,a,one


In [42]:
df['data1'].groupby([states, years])
df['data1'].groupby([states, years]).mean()

California  2005   -0.404252
            2006    0.312038
Ohio        2005   -0.972144
            2006   -0.994283
Name: data1, dtype: float64

### multiple variables (data)

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/81f9f.png)

In [54]:
df

,data1,data2,key1,key2
0,-1.013507,-0.304265,a,one
1,-0.404252,-0.351571,a,two
2,0.312038,0.178852,b,one
3,-0.930780,-0.465142,b,two
4,-0.994283,-1.118791,a,one


In [55]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.804014,-0.591542
b,-0.309371,-0.143145


In [56]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -1.003895 -0.711528
     two  -0.404252 -0.351571
b    one   0.312038  0.178852
     two  -0.930780 -0.465142

In [57]:
df.groupby(['key1', 'key2']).size() ###group size

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

###  Iterating Over Groups

In [58]:
df
grouped = df.groupby(df['key1'])
grouped

,data1,data2,key1,key2
0,-1.013507,-0.304265,a,one
1,-0.404252,-0.351571,a,two
2,0.312038,0.178852,b,one
3,-0.930780,-0.465142,b,two
4,-0.994283,-1.118791,a,one


In [66]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -1.013507 -0.304265    a  one
1 -0.404252 -0.351571    a  two
4 -0.994283 -1.118791    a  one
b
      data1     data2 key1 key2
2  0.312038  0.178852    b  one
3 -0.930780 -0.465142    b  two


In [69]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -1.013507 -0.304265    a  one
4 -0.994283 -1.118791    a  one
('a', 'two')
      data1     data2 key1 key2
1 -0.404252 -0.351571    a  two
('b', 'one')
      data1     data2 key1 key2
2  0.312038  0.178852    b  one
('b', 'two')
     data1     data2 key1 key2
3 -0.93078 -0.465142    b  two


### Group with functions

In [78]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.389908,0.041529,0.090625,-1.222478,1.693066
Steve,-1.171313,-0.207604,-0.329637,0.786079,-1.056843
Wes,1.146016,NaN,NaN,0.571388,0.212792
Jim,0.377004,-0.137962,-0.413814,-0.892922,-0.280501
Travis,0.674073,0.062894,-1.881284,-1.244068,-1.020265


In [80]:
people.groupby(len).sum() 

,a,b,c,d,e
3,1.133113,-0.096433,-0.323190,-1.544012,1.625357
5,-1.171313,-0.207604,-0.329637,0.786079,-1.056843
6,0.674073,0.062894,-1.881284,-1.244068,-1.020265


In [76]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [77]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -2.243486  0.507068  0.921179 -0.635934 -1.286943
  two  0.328436 -0.027091 -0.930685  1.180461 -1.804154
5 one -0.098654  0.547121 -0.692816 -0.535822 -1.329415
6 two  0.960792 -0.587522 -1.169046 -1.354539 -1.314214

###  Grouping by Index Levels 

In [82]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'], 
                                     [1, 3, 5, 1, 3]], 
                                    names=['cty', 'tenor'])
columns
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

MultiIndex(levels=[['JP', 'US'], [1, 3, 5]],
           labels=[[1, 1, 1, 0, 0], [0, 1, 2, 0, 1]],
           names=['cty', 'tenor'])

cty          US                            JP          
tenor         1         3         5         1         3
0      0.780396 -2.306282  1.365955 -0.535688  0.839483
1      0.075790  0.251917  0.285795  0.702377  2.736588
2     -0.452898  0.703056  1.897720 -1.105935  0.832106
3      0.497096 -0.663046  0.346437  0.771534 -1.385561

In [83]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


- A list or array of values that is the same length as the axis being grouped
- A value indicating a column name in a DataFrame
- A dict or Series giving a correspondence between the values on the axis being
grouped and the group names
- A function to be invoked on the axis index or the individual labels in the index

### `Groupby` Summary

![](http://oydgk2hgw.bkt.clouddn.com/pydata-book/81f9f.png)